In [344]:
import tensorflow as tf
import numpy as np
import pandas as pd
import json

In [345]:
with open('chat_data.json') as f:
    content = json.load(f)

In [346]:
content['intents'][0],len(content['intents'])

({'tag': 'Annoying',
  'patterns': ['you are annoying me so much',
   "you're incredibly annoying",
   'I find you annoying',
   'you are annoying',
   "you're so annoying",
   'how annoying you are',
   'you annoy me',
   'you are annoying me',
   'you are irritating',
   'you are such annoying',
   "you're too annoying",
   'you are very annoying'],
  'responses': ['Was going to say the same about you',
   'Ill try to fix that',
   'Ok thanks for the feedback.'],
  'context_set': ''},
 21)

In [347]:
labels=[]
doc=[]
responses = dict()
exclude = [",","?","!","."]

In [348]:
for intent in content['intents']:
    for pattern in intent['patterns']:
       
        doc.append([pattern,intent['tag']])
    
    responses["{}".format(intent['tag'])] = intent['responses']    
        
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        

        

In [349]:
doc = np.array(doc)
doc[:5],labels[:5]

(array([['you are annoying me so much', 'Annoying'],
        ["you're incredibly annoying", 'Annoying'],
        ['I find you annoying', 'Annoying'],
        ['you are annoying', 'Annoying'],
        ["you're so annoying", 'Annoying']], dtype='<U39'),
 ['Annoying', 'greeting', 'help', 'bye', 'Boring'])

In [350]:
df =pd.DataFrame({'Text':doc[:,0],'Intent':doc[:,1]})

In [351]:
df

,Text,Intent
0,you are annoying me so much,Annoying
1,you're incredibly annoying,Annoying
2,I find you annoying,Annoying
3,you are annoying,Annoying
4,you're so annoying,Annoying
...,...,...
285,Joke please,jokes
286,i want some jokes,jokes
287,joke,jokes
288,make me laugh,jokes


In [352]:
df['Text'] = df['Text'].apply(lambda x: x.lower())
df['Text'] = df['Text'].apply(lambda x: x.replace("?",''))

In [353]:
df

,Text,Intent
0,you are annoying me so much,Annoying
1,you're incredibly annoying,Annoying
2,i find you annoying,Annoying
3,you are annoying,Annoying
4,you're so annoying,Annoying
...,...,...
285,joke please,jokes
286,i want some jokes,jokes
287,joke,jokes
288,make me laugh,jokes


In [354]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import Dense,LSTM,Embedding,Input
from tensorflow.keras.models import Sequential

In [355]:
length=[]
length.append([len(i.split()) for i in df['Text']])
max_len = np.max(length)
max_len

7

### Preprocessing Labels

In [356]:
dummy = pd.get_dummies(df['Intent'],dtype='float32')
dummy

,Annoying,Birthday,Boring,Bot_identity,acquintance,bad_language,bye,clever,creator,greeting,...,how_are_U,jokes,marry,music,no,no_problem,offensive,questioning_bot,residence,there
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
286,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
287,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
288,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [357]:
one_hot_labels = dummy

labels_list = one_hot_labels.columns
classes = dict()
for i in range(len(labels_list)):
    classes['{}'.format(i)] = labels_list[i]


In [358]:
one_hot_labels = np.array(one_hot_labels)
one_hot_labels= one_hot_labels.reshape(-1,21)
one_hot_labels,one_hot_labels.shape

(array([[1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32),
 (290, 21))

In [359]:
classes

{'0': 'Annoying',
 '1': 'Birthday',
 '2': 'Boring',
 '3': 'Bot_identity',
 '4': 'acquintance',
 '5': 'bad_language',
 '6': 'bye',
 '7': 'clever',
 '8': 'creator',
 '9': 'greeting',
 '10': 'help',
 '11': 'how_are_U',
 '12': 'jokes',
 '13': 'marry',
 '14': 'music',
 '15': 'no',
 '16': 'no_problem',
 '17': 'offensive',
 '18': 'questioning_bot',
 '19': 'residence',
 '20': 'there'}

In [360]:
vocab_size = 5000
sentences = [x for x in doc[:,0]]
one_hot_sentences = [one_hot(d,vocab_size) for d in sentences ]
one_hot_sentences[:5]


[[1197, 2335, 4383, 1423, 559, 2834],
 [4679, 4318, 4383],
 [4041, 42, 1197, 4383],
 [1197, 2335, 4383],
 [4679, 559, 4383]]

In [361]:
one_hot_sentences = pad_sequences(one_hot_sentences,max_len,padding='post')

In [362]:
one_hot_sentences = one_hot_sentences.reshape(-1,7)
one_hot_sentences.shape

(290, 7)

In [363]:
model = Sequential([
    Input(shape=(None,)),
    Embedding(vocab_size,300),
    LSTM(64),
    Dense(100,activation='relu'),
    Dense(100,activation='relu'),
    Dense(21,activation='softmax'),
])
model.compile(
loss='CategoricalCrossentropy',
    optimizer='Adam',
    metrics=['accuracy']
)
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, None, 300)         1500000   
_________________________________________________________________
lstm_6 (LSTM)                (None, 64)                93440     
_________________________________________________________________
dense_18 (Dense)             (None, 100)               6500      
_________________________________________________________________
dense_19 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_20 (Dense)             (None, 21)                2121      
Total params: 1,612,161
Trainable params: 1,612,161
Non-trainable params: 0
_________________________________________________________________


In [364]:
model.fit(one_hot_sentences,one_hot_labels,epochs=100)

Epoch 1/100
10/10 [==============================] - 8s 37ms/step - loss: 2.9986 - accuracy: 0.2655
Epoch 2/100
10/10 [==============================] - 0s 37ms/step - loss: 2.7912 - accuracy: 0.2759
Epoch 3/100
10/10 [==============================] - 0s 35ms/step - loss: 2.5884 - accuracy: 0.2759
Epoch 4/100
10/10 [==============================] - 0s 34ms/step - loss: 2.4393 - accuracy: 0.2759
Epoch 5/100
10/10 [==============================] - 0s 39ms/step - loss: 2.2074 - accuracy: 0.2966
Epoch 6/100
10/10 [==============================] - 0s 35ms/step - loss: 2.0346 - accuracy: 0.3414
Epoch 7/100
10/10 [==============================] - 0s 32ms/step - loss: 1.8966 - accuracy: 0.3862
Epoch 8/100
10/10 [==============================] - 0s 35ms/step - loss: 1.7945 - accuracy: 0.4586
Epoch 9/100
10/10 [==============================] - 0s 34ms/step - loss: 1.6619 - accuracy: 0.4621
Epoch 10/100
10/10 [==============================] - 0s 37ms/step - loss: 1.5824 - accuracy: 0.4897

10/10 [==============================] - 0s 35ms/step - loss: 0.1012 - accuracy: 0.9793
Epoch 82/100
10/10 [==============================] - 0s 34ms/step - loss: 0.0843 - accuracy: 0.9690
Epoch 83/100
10/10 [==============================] - 0s 35ms/step - loss: 0.0798 - accuracy: 0.9724
Epoch 84/100
10/10 [==============================] - 0s 34ms/step - loss: 0.0383 - accuracy: 0.9828
Epoch 85/100
10/10 [==============================] - 0s 31ms/step - loss: 0.0701 - accuracy: 0.9759
Epoch 86/100
10/10 [==============================] - 0s 34ms/step - loss: 0.0512 - accuracy: 0.9793
Epoch 87/100
10/10 [==============================] - 0s 33ms/step - loss: 0.0681 - accuracy: 0.9724
Epoch 88/100
10/10 [==============================] - 0s 34ms/step - loss: 0.0610 - accuracy: 0.9759
Epoch 89/100
10/10 [==============================] - 0s 35ms/step - loss: 0.0447 - accuracy: 0.9828
Epoch 90/100
10/10 [==============================] - 0s 34ms/step - loss: 0.0335 - accuracy: 0.9828
Epo

In [382]:
model.save('TARS.h5')

In [365]:
def preprocess_input(input_text):
    input_text = input_text.lower()
    text = [one_hot(input_text,vocab_size)]
    text =np.array(text)
    text = pad_sequences(text,7,padding='post')
    return text

In [370]:
def predict_output(input_text):
    input_text =  preprocess_input(input_text)
    output= model.predict(input_text) 
    z=0
    for j in range(len(output[0])):  
        if output[0][j]==max(output[0]):
            x = classes['{}'.format(j)] 
            response = responses['{}'.format(x)]
            y = np.random.choice(response)           
        else:
             z+=1
    if z==len(output[0]):
        print('Bot output ========>', "I can't understand you. I think i should learn more")
        

    return y
    

In [374]:
def chat():
    print('start talking...(type "quit" to stop)')
    while True:
        inputs = input('You: ')
        if inputs.lower()=='quit':
            break
        result = predict_output(inputs.lower())
        
        print("TARS: {}".format(result))

In [396]:
chat()


start talking...(type "quit" to stop)
You: hi tars
TARS: Hi there, how can I help?
You: who are you
TARS: I'm a ChatBot
You: who made you
TARS: My master is Surendra
You: you are so smart
TARS: Thank you. I try my best.
You: i need help
TARS: I'm glad to help. What can I do for you?
You: tell ,me a joke
TARS: Why did the gym close down? It just didn't work out!
You: thank you
TARS: Good Bye
You: marry me
TARS: I know you can't mean that, but I'm flattered all the same.
You: where is your house
TARS: The virtual world is my playground. I'm always just a few clicks away.
You: when is your birthday
TARS: I'm all for celebrating my birthday at anytime but my birthday is Feb 29
You: play with me
TARS: In the virtual sense that I can, sure.
You: play some music
TARS: Sorry, I Don't have any records of that
You: no problem
TARS: It's Ok.
You: bye tars
TARS: You're Welcome! Master.
You: see ya
TARS: Nice to talk to you, Bye
You: quit
